In [166]:
from pyscf import gto, scf, mp, cc
a = 2 # bond length in a cluster
d = 2 # distance between each cluster
unit = 'b' # unit of length
na = 10  # size of a cluster (monomer)
nc = 1 # set as integer multiple of monomers
spin = 0 # spin per monomer
frozen = 0 # frozen orbital per monomer
elmt = 'H'
basis = 'sto6g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis=basis, spin=0, verbose=4, max_memory=16000)
mf = scf.UHF(mol).density_fit()
mf.kernel()

mo, _ = mf.stability()
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo, _ = mf.stability()
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mf.stability()

nfrozen = 0

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sun Dec  7 00:24:52 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 10
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.

******** <class 'pyscf.df.df.DF'> ********
auxbasis = None
max_memory = 16000
ETB for H: l = 0, exps = 0.200224856 * 2^n , n = 0..8

WARN: Even tempered Gaussians are generated as DF auxbasis for  H

init E= -2.8346686444452
  alpha nocc = 5  HOMO = -0.132922468150745  LUMO = -0.0853034175632029
  beta  nocc = 5  HOMO = -0.127330305360242  LUMO = -0.0843053568914742
cycle= 1 E= -3.97505182856933  delta_E= -1.14  |g|= 0.0526  |ddm|= 1.91
  alpha nocc = 5  HOMO = -0.175071765642576  LUMO = -0.0104605950372335
  beta  nocc = 5  HOMO = -0.175676003023685  LUMO = -0.0110869201557899
cycle= 2 E= -3.98988403701745  delta_E= -0.0148  |g|= 0.0382  |ddm|= 0.275
  alpha nocc = 5  HOMO = -0.189229576561539  LUMO = -0.00339760351573464
  beta  nocc = 5  HOMO = -0.190370796867818  LUMO = -0.00463175965477766
cycle= 3 E= -3.99640842274032  delta_E= -0.00652  |g|= 0.026  |ddm|= 0.174
  alpha nocc = 5  HOMO = -0.207916629019675  LUMO = 0.019143826706207
  beta  nocc = 5  HOMO = -0.210595176999379  LUMO

In [167]:
from pyscf import lo
import numpy as np
orbocca = mf.mo_coeff[0][:,nfrozen:np.count_nonzero(mf.mo_occ[0])]
orbloca = lo.PipekMezey(mol, orbocca).kernel()
orboccb = mf.mo_coeff[1][:,nfrozen:np.count_nonzero(mf.mo_occ[1])]
orblocb = lo.PipekMezey(mol, orboccb).kernel()
lo_coeff = [orbloca, orblocb]

lno_type = ['1h'] * 2
oa = [[[i],[]] for i in range(orbloca.shape[1])]
ob = [[[],[i]] for i in range(orblocb.shape[1])]
frag_lolist = oa + ob



******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 4.7607942068934  delta_f= 4.76079  |g|= 0.000507313  1 KF 1 Hx
macro= 2  f(x)= 4.7607942068934  delta_f= 8.88178e-16  |g|= 1.64563e-06  1 KF 1 Hx
macro X = 2  f(x)= 4.7607942068934  |g|= 1.64563e-06  4 intor 2 KF 2 Hx


******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowd

In [168]:
from collections.abc import Iterable
from pyscf.lno import ulnoccsd

mlno = ulnoccsd.ULNOCCSD_T(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=4)
thresh = 1e-4
mlno.lno_thresh = [thresh*10,thresh]
lno_thresh = mlno.lno_thresh
lno_type = ['1h','1h'] if lno_type is None else lno_type
lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
eris = None

nfrag = len(frag_lolist)
if lno_pct_occ is None:
    lno_pct_occ = [None, None]
if lno_norb is None:
    lno_norb = [[None,None]] * nfrag
mf = mlno._scf

if eris is None: eris = mlno.ao2mo()

# Loop over fragment
frag_res = [None] * nfrag
for ifrag,loidx in enumerate(frag_lolist):
    if ifrag == 0:
        if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
            orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
            lno_param = [
                [
                    {
                        'thresh': (
                            lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                            else lno_thresh[i]
                        ),
                        'pct_occ': (
                            lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                            else lno_pct_occ[i]
                        ),
                        'norb': (
                            lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                            else lno_norb[ifrag][i]
                        ),
                    } for i in [0, 1]
                ] for s in range(2)
            ]

        else:
            orbloc = lo_coeff[:,loidx]
            lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                            'norb': lno_norb[ifrag][i]} for i in [0,1]]

        lno_coeff, frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)
        # frag_res[ifrag], frag_msg = mlno.impurity_solve(mf, mo_coeff, uocc_loc, eris, frozen=frozen)

        # mo_occ = mlno.mo_occ
        # frozen, maskact = ulnoccsd.get_maskact(frozen, [mo_occ[0].size, mo_occ[1].size])
        # mcc = ulnoccsd.UCCSD(mf, mo_coeff=lno_coeff, frozen=frozen).set(verbose=mlno.verbose_imp)
        # mcc._s1e = mlno._s1e
        # mcc._h1e = mlno._h1e
        # mcc._vhf = mlno._vhf
        # if mlno.kwargs_imp is not None:
        #     mcc = mcc.set(**mlno.kwargs_imp)

        # frag_res[ifrag], t1, t2 =\
        #     ulno_ccsd(mcc, lno_coeff, uocc_loc, mo_occ, maskact, ccsd_t=True)
    else: break

LO occ proj: 1 active | 0 standby | 4 orthogonal
LO occ proj: 0 active | 0 standby | 5 orthogonal


In [169]:
# frozen = [[],[]]
nocc_a = int(sum(mf.mo_occ[0]))
actfrag_a = np.array([i for i in range(mol.nao) if i not in frozen[0]])
frzocc_a = np.array([i for i in range(nocc_a) if i in frozen[0]])
actocc_a = np.array([i for i in range(nocc_a) if i in actfrag_a])
actvir_a = np.array([i for i in range(nocc_a,mol.nao) if i in actfrag_a])
nfrzocc_a = len(frzocc_a)
nactocc_a = len(actocc_a)
nactvir_a = len(actvir_a)
nactorb_a = len(actfrag_a)
nocc_b = int(sum(mf.mo_occ[1]))
actfrag_b = np.array([i for i in range(mol.nao) if i not in frozen[1]])
frzocc_b = np.array([i for i in range(nocc_b) if i in frozen[1]])
actocc_b = np.array([i for i in range(nocc_b) if i in actfrag_b])
actvir_b = np.array([i for i in range(nocc_b,mol.nao) if i in actfrag_b])
nfrzocc_b = len(frzocc_b)
nactocc_b = len(actocc_b)
nactvir_b = len(actvir_b)
nactorb_b = len(actfrag_b)
print(nfrzocc_a,nactocc_a,nactvir_a,nactorb_a)
print(nfrzocc_b,nactocc_b,nactvir_b,nactorb_b)

4 1 1 2
5 0 1 1


In [170]:
ncas = (nactorb_a,nactorb_b)
ncore = (nfrzocc_a,nfrzocc_b)
lno_act_a = lno_coeff[0][:,ncore[0]:ncore[0]+ncas[0]]
lno_act_b = lno_coeff[1][:,ncore[1]:ncore[1]+ncas[1]]
# lno_act = np.hstack([lno_act_a,lno_act_b])
print(lno_act_a)
print(lno_act_b)

[[ 3.34575044e-05 -4.63397792e-04]
 [ 2.45213898e-04 -4.02859704e-03]
 [-1.18303302e-03  1.48147657e-02]
 [-9.87719363e-03  1.21632499e-01]
 [ 5.77219253e-02 -7.18237489e-01]
 [ 9.83011305e-01  2.51490733e-01]
 [ 5.76847754e-02 -7.17547765e-01]
 [-9.87634626e-03  1.21551953e-01]
 [-1.18032394e-03  1.48056647e-02]
 [ 2.48078913e-04 -4.12712184e-03]]
[[-7.96168441e-04]
 [ 3.03419448e-03]
 [ 2.21428135e-02]
 [-1.41616575e-01]
 [ 1.98540828e-01]
 [-9.93770393e-01]
 [ 1.98476164e-01]
 [-1.41354742e-01]
 [ 2.20895483e-02]
 [ 3.05897301e-03]]


In [171]:
s1e = mf.get_ovlp()
lno_act_aa = lno_coeff[0].T @ s1e @ lno_act_a
lno_act_ba = lno_coeff[0].T @ s1e @ lno_act_b
print(lno_act_aa)
print(lno_act_ba)

[[ 2.95445092e-17  4.64580631e-17]
 [ 1.07552856e-16  2.77555756e-17]
 [ 3.68628739e-18 -3.81639165e-17]
 [ 3.33066907e-16  4.16333634e-17]
 [ 1.00000000e+00  4.99600361e-16]
 [ 4.85722573e-16  1.00000000e+00]
 [ 3.63156906e-16 -4.19789529e-17]
 [ 8.89045781e-18 -8.32667268e-16]
 [ 1.15467531e-17  6.61038065e-16]
 [ 6.73492837e-17  1.70614458e-16]]
[[-2.15270022e-04]
 [ 1.43064334e-01]
 [ 6.58350581e-03]
 [-2.83771949e-02]
 [-9.21389497e-01]
 [-3.57397335e-01]
 [-9.99621518e-05]
 [ 4.44796350e-02]
 [ 3.73563455e-03]
 [-2.82679846e-04]]


In [45]:
def prjmo(prj,s1e,mo):
    # prj and reconstruct mo
    mo_rec = prj @ prj.T @ s1e @ mo
    return mo_rec

In [81]:
prj = np.hstack([lno_act_a[:,:0],lno_act_b[:,:]])
lno_act_b_rec = prjmo(prj,s1e,lno_act_b)
print(lno_act_b_rec.shape)
print(lno_act_b_rec-lno_act_b)

(10, 4)
[[-5.68570866e-20  2.05998413e-18  3.90312782e-18  1.95156391e-18]
 [ 3.87980795e-20 -1.02999206e-18 -1.95156391e-18 -1.02999206e-18]
 [ 4.34222970e-17 -9.99200722e-16 -1.88737914e-15 -9.43689571e-16]
 [-5.30310388e-17  1.05471187e-15  2.10942375e-15  1.11022302e-15]
 [ 3.33066907e-16  2.55351296e-15 -4.66749035e-16 -1.66533454e-15]
 [-8.88178420e-16 -1.77635684e-15  3.19080699e-16  1.11022302e-15]
 [-5.76185692e-17 -1.16573418e-15  1.66533454e-15 -3.88578059e-16]
 [ 7.63820431e-17  1.27675648e-15 -1.66533454e-15  3.88578059e-16]
 [ 7.02375602e-20  1.19262239e-18 -1.51788304e-18  3.79470760e-19]
 [-1.47383733e-19 -2.38524478e-18  3.25260652e-18 -7.58941521e-19]]


In [132]:
m = np.hstack([lno_act_aa,lno_act_ba])
u,s,v = np.linalg.svd(m)
print(u.shape,s.shape,v.shape)
print(s)

(10, 10) (7,) (7, 7)
[1.41421274 1.41419237 1.4120414  1.         0.07835238 0.00774129
 0.00152421]


In [178]:
def common_las(mf, lno_coeff, ncas, ncore, torr=1e-5):
    s1e = mf.get_ovlp()
    lno_acta = lno_coeff[0][:,ncore[0]:ncore[0]+ncas[0]]
    lno_actb = lno_coeff[1][:,ncore[1]:ncore[1]+ncas[1]]
    lno_actaa = lno_coeff[0].T @ s1e @ lno_acta
    lno_actba = lno_coeff[0].T @ s1e @ lno_actb
    m = np.hstack([lno_actaa,lno_actba])
    u,s,v = np.linalg.svd(m)
    for idx in range(mf.mol.nao):
        prj = lno_coeff[0] @ u[:,:idx]
        lno_act_b_rec = prjmo(prj,s1e,lno_act_b)
        lno_act_a_rec = prjmo(prj,s1e,lno_act_a)
        losa = abs(lno_act_b_rec-lno_act_b).max()
        losb = abs(lno_act_a_rec-lno_act_a).max()
        # print(losa, losb)
        if losa < torr and losb < torr:
            print(f"# Minimum number of SVD orbitals to span both Alpha and Beta active space: {idx}")
            print(f"# Active space loss: ({losa:.2e}, {losb:.2e})")
            break
    return lno_coeff[0] @ u[:,:idx]

In [180]:
c = common_las(mf, lno_coeff, ncas, ncore)
nclas = c.shape[1]
print(nclas)

# Minimum number of SVD orbitals to span both Alpha and Beta active space: 3
# Active space loss: (9.16e-16, 2.66e-15)
3


In [181]:
from pyscf import df, ao2mo, lib
chol_df = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
chol_df = lib.unpack_tril(chol_df).reshape(chol_df.shape[0], -1)
chol_df = chol_df.reshape((-1, mol.nao, mol.nao))
eri_ao = lib.einsum('lpr,lqs->prqs', chol_df, chol_df, optimize='optimal')
print("# Composing active space MO ERIs from AO ERIs")
# # FIX: find the space that just span active a and b
eri_clas = ao2mo.kernel(eri_ao,c,compact=False)

# Composing active space MO ERIs from AO ERIs


In [182]:
print(eri_clas.shape)

(3, 3, 3, 3)


In [183]:
from pyscf import df, ao2mo, lib
from ad_afqmc import pyscf_interface
chol_cut = 1e-5
chol_df = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
chol_df = lib.unpack_tril(chol_df).reshape(chol_df.shape[0], -1)
chol_df = chol_df.reshape((-1, mol.nao, mol.nao))
eri_ao = lib.einsum('lpr,lqs->prqs', chol_df, chol_df, optimize='optimal')
print("# Composing active space MO ERIs from AO ERIs")
# # FIX: find the space that just span active a and b
eri_clas = ao2mo.kernel(eri_ao,c,compact=False)
eri_clas = eri_clas.reshape(nclas**2,nclas**2)
print("# Decomposing MO ERIs to Cholesky vectors")
print(f"# Cholesky cutoff is: {chol_cut}")
chol_c = pyscf_interface.modified_cholesky(eri_clas,max_error=chol_cut)
chol_c = chol_c.reshape((-1, nclas, nclas))

# Composing active space MO ERIs from AO ERIs
# Decomposing MO ERIs to Cholesky vectors
# Cholesky cutoff is: 1e-05


In [184]:
print(chol_c.shape)

(5, 3, 3)


In [187]:
# <clas|a>
a2c = c.T @ s1e @ lno_act_a
b2c = c.T @ s1e @ lno_act_b
print(a2c.shape)
print(b2c.shape)

(3, 2)
(3, 1)


In [ ]:
a2c = c.T @ s1e @ lno_act_a
b2c = c.T @ s1e @ lno_act_b
chol_a = lib.einsum('pr,grs,sq->gpq',a2c.T,chol_c,a2c)
chol_b = lib.einsum('pr,grs,sq->gpq',b2c.T,chol_c,b2c)
print(chol_a.shape, chol_b.shape)
eri_lno_a = lib.einsum('gpr,gqs->prqs',chol_a,chol_a)
eri_lno_b = lib.einsum('gpr,gqs->prqs',chol_b,chol_b)
print(eri_lno_b.shape)

(5, 2, 2) (5, 1, 1)
(1, 1, 1, 1)
